In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils import data
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.init
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
NUM_EPOCHS = 90
BATCH_SIZE = 128
MOMENTUM = 0.9
LR_DECAY = 0.0005
LR_INIT = 0.01
IMAGE_DIM = 227 # pixels
NUM_CLASSES = 10
writer = SummaryWriter()

In [ ]:
class Inception(nn.Module):
  def __init__(self,in_channel,out_channel_1,out_channel_2,out_channel_3,out_channel_4,**kwargs):
    #Four output channel for each parallel block of network
    super(Inception,self).__init__()

    self.p1_1=nn.Conv2d(in_channel,out_channel_1,kernel_size=1)#1x1Conv

    self.p2_1=nn.Conv2d(in_channel,out_channel_2[0],kernel_size=1)#1x1Conv
    self.p2_2=nn.Conv2d(out_channel_2[0],out_channel_2[1],kernel_size=3,padding=1)#3x3Conv

    self.p3_1=nn.Conv2d(in_channel,out_channel_3[0],kernel_size=1)#1x1Conv
    self.p3_2=nn.Conv2d(out_channel_3[0],out_channel_3[1],kernel_size=5,padding=2)#5x5Conv

    self.p4_1=nn.MaxPool2d(kernel_size=3,stride=1,padding=1)#3x3 MaxPool
    self.p4_2=nn.Conv2d(in_channel,out_channel_4,kernel_size=1)#1x1 Conv

  def forward(self,x):
    p1=F.relu(self.p1_1(x))
    p2=F.relu(self.p2_2(F.relu(self.p2_1(x))))
    p3=F.relu(self.p3_2(F.relu(self.p3_1(x))))
    p4=F.relu(self.p4_2(self.p4_1(x)))


    return torch.cat((p1,p2,p3,p4),dim=1)
    #Finally, the outputs along each path are concatenated
    #along the channel dimension and comprise the block's output.




In [ ]:
class ConvBlock(nn.Module):
    def __init__(self,in_channels,out_channels,**kwargs):
      super(ConvBlock,self).__init__()
      self.conv=nn.Conv2d(in_channels,out_channels,**kwargs)
      self.relu=nn.ReLU()

    def forward(self,x):
      x=self.conv(x)
      x=self.relu(x)
      return x

In [ ]:
class InceptionAux(nn.Module):
  def __init__(self,in_channels,num_classes):
    super(InceptionAux,self).__init__()
    self.avgpool=nn.AvgPool2d(kernel_size=5,stride=3)
    self.conv=ConvBlock(in_channels,128,kernel_size=1,stride=1,padding=0)
    self.fc=nn.Sequential(
        nn.Linear(4*4*128,1024),
        nn.ReLU(inplace=True),
        nn.Dropout2d(p=0.7),
        nn.Linear(1024,num_classes)
    )

  def forward(self,x):
    x=self.avgpool(x)
    x=self.conv(x)
    x=torch.flatten(x,1)
    return self.fc(x)

In [ ]:
class GoogleNet(torch.nn.Module):
    def __init__(self, in_channels, num_classes):
        super(GoogleNet, self).__init__()

        self.training = True

        self.conv1 = nn.Sequential(
            ConvBlock(in_channels, 64, kernel_size=7, stride=2, padding=3),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            nn.LocalResponseNorm(2)
        )

        self.conv2 = nn.Sequential(
            ConvBlock(64, 64, kernel_size=1),
            ConvBlock(64, 192, kernel_size=3, padding=1),
            nn.LocalResponseNorm(2),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

        self.inception_3a = Inception(192, 64, (96, 128), (16, 32), 32)
        self.inception_3b = Inception(256, 128, (128, 192), (32, 96), 64)
        self.maxpool_3 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.inception_4a = Inception(480, 192, (96, 208), (16, 48), 64)
        self.aux1 = InceptionAux(512, num_classes)

        self.inception_4b = Inception(512, 160, (112, 224), (24, 64), 64)
        self.inception_4c = Inception(512, 128, (128, 256), (24, 64), 64)
        self.inception_4d = Inception(512, 112, (144, 288), (32, 64), 64)
        self.aux2 = InceptionAux(528, num_classes)

        self.inception_4e = Inception(528, 256, (160, 320), (32, 128), 128)
        self.maxpool_4 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.inception_5a = Inception(832, 256, (160, 320), (32, 128), 128)
        self.inception_5b = Inception(832, 384, (192, 384), (48, 128), 128)

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.dropout = nn.Dropout2d(p=0.4)
        self.fc = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)

        x = self.inception_3a(x)
        x = self.inception_3b(x)
        x = self.maxpool_3(x)

        x = self.inception_4a(x)
        if self.training:
            out1 = self.aux1(x)

        x = self.inception_4b(x)
        x = self.inception_4c(x)
        x = self.inception_4d(x)
        if self.training:
            out2 = self.aux2(x)

        x = self.inception_4e(x)
        x = self.maxpool_4(x)

        x = self.inception_5a(x)
        x = self.inception_5b(x)

        x = self.avgpool(x)

        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)
        if self.training:
            return [x, out1, out2]
        else:
            return x

    def set_train(self):
        self.training = True

    def set_eval(self):
        self.training = False




In [ ]:
transform=transforms.Compose([#Compose는 Transform할 리스트를 구성함
    #227x227로 하는 이유는 AlexNet의 Input이 227x227이기 때문에 데이터를 Resize해서 설정해줌 왜냐하면 MNIST데이터는 28x28크기의 데이터셋이기 때문임
    transforms.Resize(227),
    transforms.ToTensor()#ToTensor: PIL image 혹은 numpy.ndarray를 Tensor로 바꿈
])
mnist_train=datasets.MNIST(root='MNIST_data/',#MNIST 데이터 다운로드 경로 설정
                        train=True,#훈련용 데이터로 다운받을건지에 대한 여부 True시 훈련 데이터로 다운로드 받음
                        download=True,
                        transform=transform)
mnist_test=datasets.MNIST(root='MNIST_data/',#MNIST 데이터 다운로드 경로 설정
                       train=False,#훈련용 데이터로 다운 받을건지에 대한 여부 FALSE로 설정하였음으로 훈련용 데이터가 아닌 테스트용 데이터로 받음
                       download=True,
                       transform=transform)

train_loader=torch.utils.data.DataLoader(dataset=mnist_train,batch_size=BATCH_SIZE,shuffle=True,drop_last=True)
test_loader=torch.utils.data.DataLoader(dataset=mnist_test,batch_size=BATCH_SIZE,shuffle=True,drop_last=True)

In [ ]:
model=GoogleNet(1,10).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=MOMENTUM)

In [ ]:
from torchsummary import summary as summary_

summary_(model,(1,224,224),BATCH_SIZE)

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:1347: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [128, 64, 112, 112]           3,200
              ReLU-2        [128, 64, 112, 112]               0
         ConvBlock-3        [128, 64, 112, 112]               0
         MaxPool2d-4          [128, 64, 56, 56]               0
 LocalResponseNorm-5          [128, 64, 56, 56]               0
            Conv2d-6          [128, 64, 56, 56]           4,160
              ReLU-7          [128, 64, 56, 56]               0
         ConvBlock-8          [128, 64, 56, 56]               0
            Conv2d-9         [128, 192, 56, 56]         110,784
             ReLU-10         [128, 192, 56, 56]               0
        ConvBlock-11         [128, 192, 56, 56]               0
LocalResponseNorm-12         [128, 192, 56, 56]               0
        MaxPool2d-13         [128, 192, 28, 28]               0
           Conv2d-14          [128, 64,

In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # enumberate() : 인덱스와 원소로 이루어진 튜플(tuple)을 만들어줌
        target = target.type(torch.LongTensor)
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() # 항상 backpropagation 하기전에 미분(gradient)을 zero로 만들어주고 시작해야 한다.
        output = model(data)
        loss = criterion(output, target) # criterion = loss_fn
        writer.add_scalar("Loss/GoogleNet train",loss,epoch)
        loss.backward() # Computes the gradient of current tensor w.r.t. graph leaves
        optimizer.step() # step() : 파라미터를 업데이트함
        if (batch_idx + 1) % 30 == 0:
            print("Train Epoch:{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [ ]:
import torch.nn.functional as F
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target).item()
            writer.add_scalar("Loss/GoogleNet test",test_loss/len(test_loader),epoch)
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

        test_loss /= len(test_loader.dataset)  # -> mean
        print("\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
            test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))
        print('='*50)


In [ ]:
epochs=15
for epoch in range(1,epochs+1):
  train(model,device,train_loader,optimizer,epoch)



writer.flush()
writer.close()

NameError: name 'train' is not defined